In [ ]:
import torch
from torch import nn
from label_map import METRIC_NAME, METRIC_FUNC
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam

In [ ]:
class LearnWts(nn.Module):
    def __init__(self):
        super(LearnWts,self).__init__()
        self.int_med = 2
        self.num_models = 4*2
        self.linear = nn.Linear(self.num_models,self.int_med)
    
    def forward(self,inp):
        inp = self.linear(torch.log(inp))
        return inp

In [ ]:
net = LearnWts()
optim = Adam(net.parameters(), lr = 0.1)
loss_fn = nn.CrossEntropyLoss()

files = ["train_1.npy","train_2.npy","train_3.npy", "train_4.npy"]
gold = torch.Tensor(np.load("golds.npy")).long()

In [ ]:
def predict_average(score, gold):
    mmeta = [0]
    import pdb
    predictions =  np.argmax(score, axis=1).tolist()
    metrics = {}
    for mm in mmeta:
        metric_name = METRIC_NAME[mm]
        metric_func = METRIC_FUNC[mm]
        if mm < 3:
            metric = metric_func(predictions, gold)
        else:
            metric = metric_func(score.squeeze(), gold)
        metrics[metric_name] = metric
    print(metrics)

logits = []
for fi in files:
    ind_logits = np.load(fi)
    if len(logits) == 0:
        logits = ind_logits
    else:
        logits = np.concatenate((logits, ind_logits), axis = 1)

logits = np.array(logits)
inpt = torch.Tensor(logits)
total_epochs = 500
for epoch in range(total_epochs):
    print("Epoch: " + str(epoch))
    predictions = net(inpt)
    loss = loss_fn(predictions, gold)
    print("Loss: " + str(loss))
    optim.zero_grad()
    loss.backward()
    optim.step()
    score = predictions.detach().cpu().numpy()
    predict_average(score, gold)
torch.save(net.state_dict(),"model.pt")